# Init

In [2]:
# import tensorflow as tf
import comet_ml
import gc
import multiprocessing as mp
import numpy as np
import torch
import os
import pytorch_lightning as pl
import spacy
import sentence_transformers
import torch.nn.functional as F
import torch.optim as optim
import shutil
import pandas as pd

from collections import OrderedDict, defaultdict
from spacy.lang.en import English
from argparse import Namespace
from scipy.sparse import coo_matrix
from tqdm.auto import tqdm
from datetime import datetime
from operator import itemgetter
from sklearn import preprocessing
from sentence_transformers import SentenceTransformer
from torch import nn
from torch.nn import TransformerEncoder, TransformerEncoderLayer
from torch.utils.data.dataset import random_split
from torch.nn.utils.rnn import pad_sequence, pack_padded_sequence, pad_packed_sequence
from torch.utils.data import Dataset, DataLoader
from transformers import BertModel, BertTokenizer, GPT2Model, GPT2Tokenizer, RobertaTokenizer, RobertaModel, XLNetTokenizer, XLNetModel

# working directory
ROOT_DIR = '.'
DATA_DIR = f'{ROOT_DIR}/data'
CHECKPOINT_DIR = 'd:/checkpoints/earnings-call'
CHECKPOINT_TEMP_DIR = f'{ROOT_DIR}/checkpoint/earnings-call/temp'

print(f'ROOT_DIR: {ROOT_DIR}')
print(f'DATA_DIR: {DATA_DIR}')
print(f'CHECKPOINT_DIR: {CHECKPOINT_DIR}')

# COMET API KEY
COMET_API_KEY = 'tOoHzzV1S039683RxEr2Hl9PX'

# set random seed
np.random.seed(42)
torch.manual_seed(42);
torch.backends.cudnn.deterministic = False;
torch.backends.cudnn.benchmark = True;

# set device 'cuda' or 'cpu'
if torch.cuda.is_available():
    n_cuda = torch.cuda.device_count();
    
    def log_gpu_memory(verbose=False):
        torch.cuda.empty_cache()
        if verbose:
            for _ in range(n_cuda):
                print(f'GPU {_}:')
                print(f'{torch.cuda.memory_summary(_, abbreviated=True)}')
        else:
            for _ in range(n_cuda):
                memory_total = torch.cuda.get_device_properties(_).total_memory/(1024**3)
                memory_allocated = torch.cuda.memory_allocated(_)/(1024**3)
                print(f'GPU {_}: {memory_allocated: .2f}/{memory_total: .2f} (GB)')
            
    print(f'\n{n_cuda} GPUs found:');
    for _ in range(n_cuda):
        globals()[f'cuda{_}'] = torch.device(f'cuda:{_}');
        print(f'    {torch.cuda.get_device_name(_)} (cuda{_})');
        
    print('\nGPU memory:');
    log_gpu_memory();
else:
    print('GPU NOT enabled');
    
cpu = torch.device('cpu');
n_cpu = int(mp.cpu_count()/2);

print(f'\nCPU count (physical): {n_cpu}');

ROOT_DIR: .
DATA_DIR: ./data
CHECKPOINT_DIR: d:/checkpoints/earnings-call

2 GPUs found:
    GeForce RTX 2080 Ti (cuda0)
    GeForce RTX 2080 Ti (cuda1)

GPU memory:
GPU 0:  0.00/ 11.00 (GB)
GPU 1:  0.00/ 11.00 (GB)

CPU count (physical): 16


# Base

## helpers

In [8]:
# helper: refresh cuda memory
def refresh_cuda_memory():
    """
    Re-allocate all cuda memory to help alleviate fragmentation
    """
    # Run a full garbage collect first so any dangling tensors are released
    gc.collect()

    # Then move all tensors to the CPU
    for obj in gc.get_objects():
        if isinstance(obj, torch.Tensor) and obj.device!=cpu:
            obj.data = torch.empty(0)
            if isinstance(obj, torch.nn.Parameter) and obj.grad is not None:
                obj.grad.data = torch.empty(0)

    # Now empty the cache to flush the allocator
    torch.cuda.empty_cache()

# helper: flush chpt
def refresh_ckpt():
    '''
    move all `.ckpt` files to `/temp`
    '''
    # create ckpt_dir if not exists
    if not os.path.exists(CHECKPOINT_DIR):
        os.makedirs(CHECKPOINT_DIR)
    
    # create ckpt_temp_dir if not exists
    if not os.path.exists(CHECKPOINT_TEMP_DIR):
        os.makedirs(CHECKPOINT_TEMP_DIR)
    
    for name in os.listdir(CHECKPOINT_DIR):
        if name.endswith('.ckpt'):
            shutil.move(f'{CHECKPOINT_DIR}/{name}', f'{CHECKPOINT_DIR}/temp/{name}')

# helpers: load targets
def load_targets(targets_name):
    if 'targets_df' not in globals():
        print(f'Loading targets...@{Now()}')
        globals()['targets_df'] = pd.read_feather(f'{DATA_DIR}/{targets_name}.feather')
        print(f'Loading finished. @{Now()}')
        
# helpers: load preembeddings
def load_preembeddings(preembedding_type):
    if 'preembeddings' not in globals():
        print(f'Loading preembeddings...@{Now()}')
        globals()['preembeddings'] = torch.load(f"{DATA_DIR}/embeddings/preembeddings_{preembedding_type}.pt")
        print(f'Loading finished. @{Now()}')
        
# helpers: load split_df
def load_split_df(roll_type):
    split_df = pd.read_csv(f'{DATA_DIR}/split_dates.csv')
    globals()['split_df'] = split_df.loc[split_df.roll_type==roll_type]

## `train`

In [9]:
# loop one
def train_one(Model, window_i, model_hparams, train_hparams):
    
    # set window
    model_hparams.update({'window': split_df.iloc[window_i].window})
    
    # init model
    model = Model(model_hparams)

    # get model type
    train_hparams['task_type'] = model.task_type
    train_hparams['feature_type'] = model.feature_type
    train_hparams['model_type'] = model.model_type
    train_hparams['attn_type'] = model.attn_type

    # checkpoint
    ckpt_prefix = f"{train_hparams['model_type']}_{model_hparams['window']}_"
    checkpoint_callback = pl.callbacks.ModelCheckpoint(
        verbose=False,
        mode='min',
        monitor='val_loss',
        filepath=CHECKPOINT_DIR,
        prefix=ckpt_prefix,
        save_top_k=train_hparams['save_top_k'],
        period=train_hparams['checkpoint_period'])

    # logger
    logger = pl.loggers.CometLogger(
        api_key=COMET_API_KEY,
        save_dir='/data/logs',
        project_name='earnings-call',
        experiment_name=model_hparams['window'],
        workspace='amiao',
        display_summary_level=0)

    # early stop
    early_stop_callback = pl.callbacks.EarlyStopping(
        monitor='val_loss',
        min_delta=0,
        patience=train_hparams['early_stop_patience'],
        verbose=False,
        mode='min')

    # trainer
    trainer = pl.Trainer(gpus=-1, 
                         checkpoint_callback=checkpoint_callback, 
                         early_stop_callback=early_stop_callback,
                         overfit_batches=train_hparams['overfit_batches'], 
                         row_log_interval=train_hparams['row_log_interval'],
                         val_check_interval=train_hparams['val_check_interval'], 
                         progress_bar_refresh_rate=2, 
                         distributed_backend='dp', 
                         accumulate_grad_batches=train_hparams['accumulate_grad_batches'],
                         min_epochs=train_hparams['min_epochs'],
                         max_epochs=train_hparams['max_epochs'], 
                         max_steps=train_hparams['max_steps'], 
                         logger=logger)

    # delete unused hparam
    if model.model_type=='mlp': model_hparams.pop('final_tdim',None)
    if model.feature_type=='fin-ratio': 
        model_hparams.pop('max_seq_len',None)
        model_hparams.pop('n_layers_encoder',None)
        model_hparams.pop('n_head_encoder',None)
        model_hparams.pop('d_model',None)
        model_hparams.pop('dff',None)
    if model.feature_type=='text': 
        model_hparams.pop('normalize_layer',None)
        model_hparams.pop('normalize_batch',None)
    if model.attn_type!='mha': model_hparams.pop('n_head_decoder',None)

    # add n_model_params
    train_hparams['n_model_params'] = sum(p.numel() for p in model.parameters())

    # upload hparams
    logger.experiment.log_parameters(model_hparams)
    logger.experiment.log_parameters(train_hparams)

    # refresh GPU memory
    refresh_cuda_memory()

    # fit and test
    try:
        # train the model
        trainer.fit(model)

        # test on the best model
        trainer.test(ckpt_path='best')

    except RuntimeError as e:
        raise e
    finally:
        del model, trainer
        refresh_cuda_memory()
        logger.finalize('finished')

## `Dataset`

> `val` and `train` are of same period

In [10]:
# Dataset: Txt + Fin-ratio
class CCDataset(Dataset):
    
    def __init__(self, split_window, split_type, text_in_dataset, roll_type, print_window, preembeddings, targets_df, split_df, valid_transcriptids=None):
        '''
        Args:
            preembeddings (from globals): list of embeddings. Each element is a tensor (S, E) where S is number of sentences in a call
            targets_df (from globals): DataFrame of targets variables.
            split_df (from globals):
            split_window: str. e.g., "roll-09"
            split_type: str. 'train' or 'test'
            text_only: only output CAR and transcripts if true, otherwise also output financial ratios
            transcriptids: list. If provided, only the given transcripts will be used in generating the Dataset. `transcriptids` is applied **on top of** `split_window` and `split_type`
        '''

        # get split dates from `split_df`
        _, train_start, train_end, test_start, test_end, _ = tuple(split_df.loc[(split_df.window==split_window) & (split_df.roll_type==roll_type)].iloc[0])
        # print current window
        if print_window:
            print(f'Current window: {split_window} ({roll_type}) \n(train: {train_start} to {train_end}) (test: {test_start} to {test_end})')
        
        train_start = datetime.strptime(train_start, '%Y-%m-%d').date()
        train_end = datetime.strptime(train_end, '%Y-%m-%d').date()
        test_start = datetime.strptime(test_start, '%Y-%m-%d').date()
        test_end = datetime.strptime(test_end, '%Y-%m-%d').date()
        
        # select valid transcriptids (preemb_keys) according to split dates 
        if split_type=='train':
            transcriptids = targets_df[targets_df.ciq_call_date.between(train_start, train_end)].transcriptid.sample(frac=1, random_state=42).tolist()
            transcriptids = transcriptids[:int(len(transcriptids)*0.9)]
            
        if split_type=='val':
            transcriptids = targets_df[targets_df.ciq_call_date.between(train_start, train_end)].transcriptid.sample(frac=1, random_state=42).tolist()
            transcriptids = transcriptids[int(len(transcriptids)*0.9):]

        elif split_type=='test':
            transcriptids = targets_df[targets_df.ciq_call_date.between(test_start, test_end)].transcriptid.tolist()

        self.valid_preemb_keys = set(transcriptids).intersection(set(preembeddings.keys()))
        
        if valid_transcriptids is not None:
            self.valid_preemb_keys = self.valid_preemb_keys.intersection(set(valid_transcriptids))
        
        # self attributes
        self.text_in_dataset = text_in_dataset
        if text_in_dataset:
            self.preembeddings = preembeddings
        self.targets_df = targets_df
        self.sent_len = sorted([(k, preembeddings[k].shape[0]) for k in self.valid_preemb_keys], key=itemgetter(1))
        self.train_start = train_start
        self.train_end = train_end
        self.test_start = test_start
        self.test_end = test_end
        self.n_samples = len(self.sent_len)
        self.split_window = split_window
        self.split_type = split_type
        
    def __len__(self):
        return (len(self.valid_preemb_keys))
    
    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()
            
        transcriptid = self.sent_len[idx][0]
        targets = self.targets_df[self.targets_df.transcriptid==transcriptid].iloc[0]
        
        # all of the following targests are
        # of type `numpy.float64`
        docid = targets.docid
        
        sue = targets.sue
        sest = targets.sest
        car_0_30 = targets.car_0_30
        car_0_30_norm = targets.car_0_30_norm
        
        alpha = targets.alpha
        volatility = targets.volatility
        mcap = targets.mcap/1e6
        bm = targets.bm
        roa = targets.roa
        debt_asset = targets.debt_asset
        numest = targets.numest
        smedest = targets.smedest
        sstdest = targets.sstdest
        car_m1_m1 = targets.car_m1_m1
        car_m2_m2 = targets.car_m2_m2
        car_m30_m3 = targets.car_m30_m3
        volume = targets.volume
        revision = targets.revision
        revision_norm = targets.revision_norm
        inflow = targets.inflow
        inflow_norm = targets.inflow_norm
        
        if self.text_in_dataset:
            # inputs: preembeddings
            embeddings = self.preembeddings[transcriptid]
            
            return car_0_30, car_0_30_norm, inflow, inflow_norm, revision, revision_norm, \
                   transcriptid, embeddings, \
                   torch.tensor([alpha, car_m1_m1, car_m2_m2, car_m30_m3, sest, sue, numest, sstdest, smedest, mcap, roa, bm, debt_asset, volatility, volume], dtype=torch.float32)
        else:
            return docid, \
                   torch.tensor(car_0_30,dtype=torch.float32), \
                   torch.tensor(car_0_30_norm,dtype=torch.float32), \
                   torch.tensor([alpha, car_m1_m1, car_m2_m2, car_m30_m3, sest, sue, numest, sstdest, smedest, mcap, roa, bm, debt_asset, volatility, volume], dtype=torch.float32)

## `Model`

In [11]:
# Model: position encoder
class PositionalEncoding(nn.Module):
    def __init__(self, d_model, dropout=0.1, max_len=5000):
        super(PositionalEncoding, self).__init__()
        self.dropout = nn.Dropout(p=dropout)

        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-np.log(10000.0) / d_model))
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        
        # pe: (max_len, 1, d_model)
        pe = pe.unsqueeze(0).transpose(0, 1)
        self.register_buffer('pe', pe)

    def forward(self, x):
        x = x + self.pe[:x.size(0), :] # (S, N, E)
        return self.dropout(x)
    
# Model: Base
class CC(pl.LightningModule):
    def __init__(self, hparams):
        super().__init__()
        
        self.hparams = Namespace(**hparams)
        # self.text_in_dataset will be filled during instanciating.
        
        global preembeddings, targets_df, split_df
        self.preembeddings = preembeddings
        self.targets_df = targets_df
        self.split_df = split_df

    # forward
    def forward(self):
        pass
    
    # loss
    def mse_loss(self, y, t):
        return F.mse_loss(y, t)
        
    # validation step
    def validation_epoch_end(self, outputs):
        mse = torch.stack([x['val_loss'] for x in outputs]).mean()
        rmse = torch.sqrt(mse)
        return {'val_loss': mse, 'log': {'val_rmse': rmse}}
    
    # test step
    def test_epoch_end(self, outputs):
        mse = torch.stack([x['test_loss'] for x in outputs]).mean()
        rmse = torch.sqrt(mse)

        return {'test_loss': mse, 'log': {'test_rmse': rmse}, 'progress_bar':{'test_rmse': rmse}}
    
    # Dataset
    def prepare_data(self):
        
        self.train_dataset = CCDataset(self.hparams.window, split_type='train', text_in_dataset=self.text_in_dataset,
                                       roll_type=self.hparams.roll_type, print_window=True, preembeddings=self.preembeddings,
                                       targets_df=self.targets_df, split_df=self.split_df)
        self.val_dataset = CCDataset(self.hparams.window, split_type='val', text_in_dataset=self.text_in_dataset,
                                     roll_type=self.hparams.roll_type, print_window=False, preembeddings=self.preembeddings,
                                       targets_df=self.targets_df, split_df=self.split_df)
        self.test_dataset = CCDataset(self.hparams.window, split_type='test', text_in_dataset=self.text_in_dataset, 
                                      roll_type=self.hparams.roll_type, print_window=False, preembeddings=self.preembeddings,
                                       targets_df=self.targets_df, split_df=self.split_df)

    # DataLoader
    def train_dataloader(self):
        # Caution:
        # - If you enable `BatchNorm`, then must set `drop_last=True`.

        collate_fn = self.collate_fn if self.text_in_dataset else None
        return DataLoader(self.train_dataset, batch_size=self.hparams.batch_size, shuffle=True, drop_last=True, num_workers=0, pin_memory=True, collate_fn=collate_fn)
    
    def val_dataloader(self):
        # Caution: 
        # - To improve the validation speed, I'll set val_batch_size to 4. 
        # - Must set `drop_last=True`, otherwise the `val_loss` tensors for different batches won't match and hence give you error.
        # - Not to set `val_batch_size` too large (e.g., 16), otherwise you'll lose precious validation data points
        
        collate_fn = self.collate_fn if self.text_in_dataset else None
        return DataLoader(self.val_dataset, batch_size=self.hparams.val_batch_size, num_workers=0, pin_memory=True, collate_fn=collate_fn, drop_last=True)

    def test_dataloader(self):
        collate_fn = self.collate_fn if self.text_in_dataset else None
        return DataLoader(self.test_dataset, num_workers=0, pin_memory=True, collate_fn=collate_fn)
    
    def collate_fn(self, data):
        '''create mini-batch

        Retures:
            embeddings: tensor, (N, S, E)
            mask: tensor, (N, S)
            sue,car,selead,sest: tensor, (N,)
        '''
        
        # embeddings: (N, S, E)
        car_0_30, car_0_30_norm, inflow, inflow_norm, revision, revision_norm, \
        transcriptid, embeddings, \
        fin_ratios = zip(*data)
            
        # pad sequence
        # the number of `padding_value` is irrelevant, since we'll 
        # apply a mask in the Transformer encoder, which will 
        # eliminate the padded positions.
        valid_seq_len = [emb.shape[-2] for emb in embeddings]
        embeddings = pad_sequence(embeddings, batch_first=True, padding_value=0) # (N, T, E)

        # mask: (N, T)
        mask = torch.ones((embeddings.shape[0], embeddings.shape[1]))
        for i, length in enumerate(valid_seq_len):
            mask[i, :length] = 0
        mask = mask == 1
        
        return torch.tensor(car_0_30, dtype=torch.float32), torch.tensor(car_0_30_norm, dtype=torch.float32), \
               torch.tensor(inflow, dtype=torch.float32), torch.tensor(inflow_norm, dtype=torch.float32), \
               torch.tensor(revision, dtype=torch.float32), torch.tensor(revision_norm, dtype=torch.float32), \
               torch.tensor(transcriptid, dtype=torch.float32), embeddings.float(), mask, \
               torch.tensor(fin_ratios, dtype=torch.float32)
        
    # optimizer
    def configure_optimizers(self):
        optimizer = optim.Adam(self.parameters(), lr=self.hparams.learning_rate)
        return optimizer   

# MLP

## model

In [12]:
# MLP
class CCMLP(CC):
    def __init__(self, hparams):
        # by super().__init__, `self.hparams` will be created
        super().__init__(hparams)
        
        # attibutes
        self.model_type = 'MLP'
        self.target_type = 'car'
        self.feature_type = 'fr'
        self.attn_type = 'dotprod'
        
        self.text_in_dataset = True if self.feature_type!='fin-ratio' else False 
        
        # dropout layers
        self.dropout1 = nn.Dropout(self.hparams.dropout)
        self.dropout2 = nn.Dropout(self.hparams.dropout)
        
        # fc layers
        self.linear1 = nn.Linear(15, 64)
        self.linear2 = nn.Linear(64, 64)
        self.linear3 = nn.Linear(64, 1)
        
        # batch normalization
        if self.hparams.normalize_batch:
            self.batch_norm = nn.BatchNorm1d(15)
            
        # layer normalization
        if self.hparams.normalize_layer:
            self.layer_norm = nn.LayerNorm(15)
            
    # forward
    def forward(self, inp):
        
        # batch normalization
        if self.hparams.normalize_batch:
            inp = self.batch_norm(inp)
            
        # layer norm
        if self.hparams.normalize_layer:
            inp = self.layer_norm(inp)
        
        x_car = self.dropout1(F.relu(self.linear1(inp)))
        x_car = self.dropout2(F.relu(self.linear2(x_car)))
        y_car = self.linear3(x_car) # (N, 1)
        
        
        return y_car
    
        
    # train step
    def training_step(self, batch, idx):
        
        _, car, car_norm, fin_ratio = batch
        
        # forward
        y_car = self.forward(fin_ratio) # (N, 1)

        # compute loss
        loss_car = self.mse_loss(y_car, car_norm.unsqueeze(-1)) # ()
        
        # logging
        return {'loss': loss_car, 'log': {'train_loss': loss_car}}
        
    # validation step
    def validation_step(self, batch, idx):
        
        _, car, car_norm, fin_ratio = batch
        
        # forward
        y_car = self.forward(fin_ratio) # (N, 1)

        # compute loss
        loss_car = self.mse_loss(y_car, car_norm.unsqueeze(-1)) # ()
        
        # logging
        return {'val_loss': loss_car}    
        
    # test step
    def test_step(self, batch, idx):
        
        _, car, car_norm, fin_ratio = batch
        
        # forward
        y_car = self.forward(fin_ratio) # (N, 1)

        # compute loss
        loss_car = self.mse_loss(y_car, car_norm.unsqueeze(-1)) # ()
        
        # logging
        return {'test_loss': loss_car}  

## run

In [19]:
# choose Model
Model = CCMLP

# model hparams
model_hparams = {
    'preembedding_type': 'all_sbert_roberta_nlistsb_encoded', # key!
    'targets_name': 'f_sue_keydevid_car_finratio_vol_transcriptid_sim_inflow_revision_text_norm', # key!
    'normalize_batch': True, # key!
    'normalize_layer': False, # key!
    'normalize_target': True, # key!
    'roll_type': '3y',
    'batch_size': 128, # 128
    'val_batch_size':2,
    'learning_rate': 5e-4,
    'task_weight': 1,
    'dropout': 0.5,
}

# train hparams
train_hparams = {
    # checkpoint & log
    'note': '3y-MLP-fr-norm', # key!
    'row_log_interval': 1,
    'save_top_k': 1,
    'val_check_interval': 1.0,
    'machine': 'yu-workstation',

    # data size
    'overfit_batches': 0.0,
    'min_epochs': 10, 
    'max_epochs': 50, 
    'max_steps': None,
    'accumulate_grad_batches': 1,

    # Caution:
    # The check of patience depends on **how often you compute your val_loss** (`val_check_interval`). 
    # Say you check val every N baches, then `early_stop_callback` will compare to your latest N **baches**.
    # If you compute val_loss every N **epoches**, then `early_stop_callback` will compare to the latest N **epochs**.
    'early_stop_patience': 8,

    # Caution:
    # If set to 1, then save ckpt every 1 epoch
    # If set to 0, then save ckpt on every val!!! (if val improves)
    'checkpoint_period': 1}

# delete all existing .ckpt files
refresh_ckpt()

# load split_df
load_split_df(model_hparams['roll_type'])
    
# load targets_df
load_targets(model_hparams['targets_name'])

# load preembeddings
# you have to do this because CCDataset requires it
load_preembeddings(model_hparams['preembedding_type'])
    
# loop over 24!
for window_i in range(len(split_df))[5:6]:

    # train one window
    train_one(Model, window_i, model_hparams, train_hparams)

CometLogger will be initialized in online mode
COMET INFO: Experiment is live on comet.ml https://www.comet.ml/amiao/earnings-call/f31e81d10385410589767ec6c226cba0

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
CUDA_VISIBLE_DEVICES: [0,1]


Current window: roll-06 (3y) 
(train: 2009-04-01 to 2012-03-31) (test: 2012-04-01 to 2012-06-30)



  | Name       | Type        | Params
-------------------------------------------
0 | dropout1   | Dropout     | 0     
1 | dropout2   | Dropout     | 0     
2 | linear1    | Linear      | 1 K   
3 | linear2    | Linear      | 4 K   
4 | linear3    | Linear      | 65    
5 | batch_norm | BatchNorm1d | 30    



Epoch 00000: val_loss reached 0.72957 (best 0.72957), saving model to d:/checkpoints/earnings-call\mlp_roll-06_epoch=0.ckpt as top 1



Epoch 00001: val_loss reached 0.68039 (best 0.68039), saving model to d:/checkpoints/earnings-call\mlp_roll-06_epoch=1.ckpt as top 1


...

COMET INFO: Uploading stats to Comet before program termination (may take several seconds)


Current window: roll-06 (3y) 
(train: 2009-04-01 to 2012-03-31) (test: 2012-04-01 to 2012-06-30)


COMET INFO: Experiment is live on comet.ml https://www.comet.ml/amiao/earnings-call/f31e81d10385410589767ec6c226cba0



COMET INFO: Uploading stats to Comet before program termination (may take several seconds)


--------------------------------------------------------------------------------
TEST RESULTS
{'test_loss': tensor(0.7321, device='cuda:0'),
 'test_rmse': tensor(0.8556, device='cuda:0')}
--------------------------------------------------------------------------------



# RNN

## Model

In [ ]:
# CCGRU
class CCGRU(CC):
    def __init__(self, hparams):
        super().__init__(hparams)
        
        self.hparams = hparams
        
        # set model types
        self.task_type = 'single'
        self.feature_type = 'univariate'
        self.model_type = 'gru'
        self.attn_type = 'dotprod'
        self.text_in_dataset = True if self.feature_type!='fin-ratio' else False 
        
        # layers
        self.gru_expert = nn.GRU(hparams.d_model, hparams.rnn_hidden_size, num_layers=4, batch_first=True,
                                 dropout=0.1, bidirectional=True)
        self.dropout_expert = nn.Dropout(hparams.dropout)
        self.linear_car = nn.Linear(hparams.rnn_hidden_size*2, 1)

    # forward
    def forward(self, inp, valid_seq_len):
        # Note: inp is [N, S, E] and **already** been packed
        self.gru_expert.flatten_parameters()
        
        # if S is longer than `max_seq_len`, cut
        inp = inp[:,:self.hparams.max_seq_len,] # (N, S, E)
        valid_seq_len[valid_seq_len>self.hparams.max_seq_len] = self.hparams.max_seq_len # (N,)
        
        # RNN layers
        inp = pack_padded_sequence(inp, valid_seq_len, batch_first=True, enforce_sorted=False)
        x_expert = pad_packed_sequence(self.gru_expert(inp)[0], batch_first=True)[0][:,-1,:] # (N, E)
        
        # final FC layers
        y_car = self.linear_car(x_expert) # (N, E)
        
        return y_car
    
    # train step
    def training_step(self, batch, idx):
        
        car, transcriptid, embeddings, mask, alpha, car_m1_m1, car_m2_m2, car_m30_m3,\
        sest, sue, numest, sstdest, smedest, \
        mcap, roa, bm, debt_asset, volatility = batch
        
        # get valid seq_len
        valid_seq_len = torch.sum(~mask, -1)
        
        # forward
        y_car = self.forward(embeddings, valid_seq_len) # (N, 1)

        # compute loss
        loss_car = self.mse_loss(y_car, car.unsqueeze(-1)) # ()
        
        # logging
        return {'loss': loss_car, 'log': {'train_loss': loss_car}}
            
    # validation step
    def validation_step(self, batch, idx):
        
        car, transcriptid, embeddings, mask, alpha, car_m1_m1, car_m2_m2, car_m30_m3,\
        sest, sue, numest, sstdest, smedest, \
        mcap, roa, bm, debt_asset, volatility = batch
        
        # get valid seq_len
        valid_seq_len = torch.sum(~mask, -1)
        
        # forward
        y_car = self.forward(embeddings, valid_seq_len) # (N, 1)

        # compute loss
        loss_car = self.mse_loss(y_car, car.unsqueeze(-1)) # ()
        
        # logging
        return {'val_loss': loss_car}        
    
    # test step
    def test_step(self, batch, idx):
        
        car, transcriptid, embeddings, mask, alpha, car_m1_m1, car_m2_m2, car_m30_m3,\
        sest, sue, numest, sstdest, smedest, \
        mcap, roa, bm, debt_asset, volatility = batch
        
        # get valid seq_len
        valid_seq_len = torch.sum(~mask, -1)
        
        # forward
        y_car = self.forward(embeddings, valid_seq_len) # (N, 1)

        # compute loss
        loss_car = self.mse_loss(y_car, car.unsqueeze(-1)) # ()
        
        # logging
        return {'test_loss': loss_car}  

## run

In [ ]:
# loop over 24 windows
load_split_df()
load_targets()

# model hparams
model_hparams = {
    'preembedding_type': 'all_sbert_roberta_nlistsb_encoded', # key
    'batch_size': 8, # key
    'val_batch_size': 1, # key
    
    'max_seq_len': 1024, 
    'learning_rate': 3e-4,
    'task_weight': 1,

    'n_layers_encoder': 6,
    'n_head_encoder': 8, # optional
    'd_model': 1024,
    'rnn_hidden_size': 64,
    'final_tdim': 1024, # optional
    'dff': 2048,
    'attn_dropout': 0.1,
    'dropout': 0.5,
    'n_head_decoder': 8} # optional

# train hparams
train_hparams = {
    # checkpoint & log
    'note': 'temp',
    'checkpoint_path': 'D:\Checkpoints\earnings-call',
    'row_log_interval': 1,
    'save_top_k': 1,
    'val_check_interval': 0.25,

    # data size
    'overfit_pct': 1,
    'min_epochs': 0,
    'max_epochs': 1,
    'max_steps': None,
    'accumulate_grad_batches': 1,

    # Caution:
    # The check of patience depends on **how often you compute your val_loss** (`val_check_interval`). 
    # Say you check val every N baches, then `early_stop_callback` will compare to your latest N **baches**.
    # If you compute val_loss every N **epoches**, then `early_stop_callback` will compare to the latest N **epochs**.
    'early_stop_patience': 5,

    # Caution:
    # If set to 1, then save ckpt every 1 epoch
    # If set to 0, then save ckpt on every val!!! (if val improves)
    'checkpoint_period': 0}

# delete all existing .ckpt files
refresh_ckpt(train_hparams['checkpoint_path'])
    
# loop over 24!
for window_i in range(len(split_df)):
    # load preembeddings
    load_preembeddings(model_hparams['preembedding_type'])

    # train one window
    train_one(CCGRU, window_i, model_hparams, train_hparams)

# Transformer

## STL

In [10]:
# STL-text-MLP
class CCTransformerSTLTxt(CC):
        
    def __init__(self, hparams):
        hparams = Namespace(**hparams)
        super().__init__(hparams)
        
        self.hparams = hparams
        
        # specify model type
        self.task_type = 'single'
        self.feature_type = 'text'
        self.attn_type = 'dotprod'
        self.model_type = 'transformer'
        self.text_in_dataset = True if self.feature_type!='fin-ratio' else False 
        
        # positional encoding
        self.encoder_pos = PositionalEncoding(hparams.d_model, hparams.attn_dropout)
        
        # encoder layers for input, expert, nonexpert
        encoder_layers_expert = nn.TransformerEncoderLayer(hparams.d_model, hparams.n_head_encoder, hparams.dff, hparams.attn_dropout)
        
        # atten layers for SUE, CAR, SELEAD, SEST
        self.attn_layers_car = nn.Linear(hparams.d_model, 1)
        self.attn_dropout = nn.Dropout(hparams.attn_dropout)
        
        # Build Encoder and Decoder
        self.encoder_expert = nn.TransformerEncoder(encoder_layers_expert, hparams.n_layers_encoder)
        
        # linear layer to produce final result
        self.linear_car_1 = nn.Linear(hparams.d_model, hparams.d_model)
        self.linear_car_2 = nn.Linear(hparams.d_model, hparams.final_tdim)
        self.linear_car_3 = nn.Linear(hparams.final_tdim, 1)
        
        self.dropout_1 = nn.Dropout(hparams.dropout)
        self.dropout_2 = nn.Dropout(hparams.dropout)
        
    # forward
    def forward(self, inp, src_key_padding_mask):
        bsz, embed_dim = inp.size(0), inp.size(2)
        
        # if S is longer than max_seq_len, cut
        inp = inp[:,:self.hparams.max_seq_len,] # (N, S, E)
        src_key_padding_mask = src_key_padding_mask[:,:self.hparams.max_seq_len] # (N, S)
        
        inp = inp.transpose(0, 1) # (S, N, E)
        
        # positional encoding
        x = self.encoder_pos(inp) # (S, N, E)
        
        # encode
        x_expert = self.encoder_expert(x, src_key_padding_mask=src_key_padding_mask).transpose(0,1) # (N, S, E)
        
        # decode with attn
        x_attn = self.attn_dropout(F.softmax(self.attn_layers_car(x_expert), dim=1)) # (N, S, 1)
        y_car = torch.bmm(x_expert.transpose(-1,-2), x_attn).squeeze(-1) # (N, E)
        
        # final linear layer
        y_car = self.dropout_1(F.relu(self.linear_car_1(y_car)))
        y_car = self.dropout_2(F.relu(self.linear_car_2(y_car)))
        y_car = self.linear_car_3(y_car) # (N,1)
        
        # final output
        return y_car
    
    # traning step
    def training_step(self, batch, idx):
        
        car, transcriptid, embeddings, mask, alpha, car_m1_m1, car_m2_m2, car_m30_m3,\
        sest, sue, numest, sstdest, smedest, \
        mcap, roa, bm, debt_asset, volatility, volume, inflow, revision = batch
        
        # get batch size
        bsz = sue.size(0)
        
        # forward
        y_car = self.forward(embeddings, mask) # (N, 1)

        # compute loss
        loss_car = self.mse_loss(y_car, car.unsqueeze(-1)).unsqueeze(-1) # (1,)
        
        # logging
        return {'loss': loss_car, 'log': {'train_loss': loss_car}}
        
    # validation step
    def validation_step(self, batch, idx):
        car, transcriptid, embeddings, mask, alpha, car_m1_m1, car_m2_m2, car_m30_m3,\
        sest, sue, numest, sstdest, smedest, \
        mcap, roa, bm, debt_asset, volatility, volume, inflow, revision = batch
        
        # get batch size
        bsz = sue.size(0)

        # forward
        y_car = self.forward(embeddings, mask) # (N, 1)

        # compute loss
        loss_car = self.mse_loss(y_car, car.unsqueeze(-1)).unsqueeze(-1) # (1,)

        # logging
        return {'val_loss': loss_car}

    # test step
    def test_step(self, batch, idx):
        car, transcriptid, embeddings, mask, alpha, car_m1_m1, car_m2_m2, car_m30_m3,\
        sest, sue, numest, sstdest, smedest, \
        mcap, roa, bm, debt_asset, volatility, volume, inflow, revision = batch
        
        # get batch size
        bsz = sue.size(0)

        # forward
        y_car = self.forward(embeddings, mask) # (N, 1)

        # compute loss
        loss_car = self.mse_loss(y_car, car.unsqueeze(-1)).unsqueeze(-1) # (1,)

        # logging
        return {'test_loss': loss_car}  

In [11]:
# STL-text-fr-MLP
class CCTransformerSTLTxtFr(CC):
    def __init__(self, hparams):
        # `self.hparams` will be created by super().__init__
        super().__init__(hparams)
        
        # specify model type
        self.model_type = 'MLP'
        self.target_type = 'car'
        self.feature_type = 'txt+fr'
        self.attn_type = 'dotprod'
        self.text_in_dataset = True if self.feature_type!='fr' else False 

        self.n_covariate = 15
        
        # positional encoding
        self.encoder_pos = PositionalEncoding(self.hparams.d_model, self.hparams.attn_dropout)
        
        # encoder layers for input, expert, nonexpert
        encoder_layers_expert = nn.TransformerEncoderLayer(self.hparams.d_model, self.hparams.n_head_encoder, self.hparams.dff, self.hparams.attn_dropout)
        
        # atten layers for SUE, CAR, SELEAD, SEST
        self.attn_layers_car = nn.Linear(self.hparams.d_model, 1)
        self.attn_dropout_1 = nn.Dropout(self.hparams.attn_dropout)
        
        # Build Encoder and Decoder
        self.encoder_expert = nn.TransformerEncoder(encoder_layers_expert, self.hparams.n_layers_encoder)
        
        # linear layer to produce final result
        self.linear_car_1 = nn.Linear(self.hparams.d_model, self.hparams.d_model)
        self.linear_car_2 = nn.Linear(self.hparams.d_model, self.hparams.final_tdim)
        self.linear_car_3 = nn.Linear(self.hparams.final_tdim+self.n_covariate, self.hparams.final_tdim+self.n_covariate)
        self.linear_car_4 = nn.Linear(self.hparams.final_tdim+self.n_covariate, self.hparams.final_tdim+self.n_covariate)
        self.linear_car_5 = nn.Linear(self.hparams.final_tdim+self.n_covariate, 1)
        
        # dropout for final fc layers
        self.final_dropout_1 = nn.Dropout(self.hparams.dropout)
        self.final_dropout_2 = nn.Dropout(self.hparams.dropout)
        self.final_dropout_3 = nn.Dropout(self.hparams.dropout)
        
        # layer normalization
        if self.hparams.normalize_layer:
            self.layer_norm = nn.LayerNorm(self.hparams.final_tdim+self.n_covariate)
            
        # batch normalization
        if self.hparams.normalize_batch:
            self.batch_norm = nn.BatchNorm1d(self.n_covariate)

    # forward
    def forward(self, embeddings, src_key_padding_mask, fin_ratios):
        
        bsz, embed_dim = embeddings.size(0), embeddings.size(2)
        
        # if S is longer than max_seq_len, cut
        embeddings = embeddings[:,:self.hparams.max_seq_len,] # (N, S, E)
        src_key_padding_mask = src_key_padding_mask[:,:self.hparams.max_seq_len] # (N, S)
        
        embeddings = embeddings.transpose(0, 1) # (S, N, E)
        
        # positional encoding
        x = self.encoder_pos(embeddings) # (S, N, E)
        
        # encode
        x_expert = self.encoder_expert(x, src_key_padding_mask=src_key_padding_mask).transpose(0,1) # (N, S, E)
        
        # decode with attn
        x_attn = self.attn_dropout_1(F.softmax(self.attn_layers_car(x_expert), dim=1)) # (N, S, 1)
        x_expert = torch.bmm(x_expert.transpose(-1,-2), x_attn).squeeze(-1) # (N, E)
        
        # mix with covariate
        x_expert = self.final_dropout_1(F.relu(self.linear_car_1(x_expert))) # (N, E)
        x_expert = F.relu(self.linear_car_2(x_expert)) # (N, final_tdim)
        
        # batch normalization
        if self.hparams.normalize_batch:
            fin_ratios = self.batch_norm(fin_ratios)
        
        x_final = torch.cat([x_expert, fin_ratios], dim=-1) # (N, X + final_tdim) where X is the number of covariate (n_covariate)

        # layer normalization
        if self.hparams.normalize_layer:
            x_final = self.layer_norm(x_final)
            
        # final FC
        x_final = self.final_dropout_2(F.relu(self.linear_car_3(x_final))) # (N, X + final_tdim)
        x_final = self.final_dropout_3(F.relu(self.linear_car_4(x_final))) # (N, X + final_tdim)
        y_car = self.linear_car_5(x_final) # (N,1)
        
        # final output
        return y_car
    
    # traning step
    def training_step(self, batch, idx):
        car, car_norm, inflow, inflow_norm, revision, revision_norm, \
        transcriptid, embeddings, mask, \
        fin_ratios = batch
        
        # get batch size
        bsz = fin_ratios.size(0)
        
        # forward
        y_car = self.forward(embeddings, mask, fin_ratios) # (N, 1)

        # compute loss
        loss_car = self.mse_loss(y_car, car_norm.unsqueeze(-1)).unsqueeze(-1) # (1,)
        
        # logging
        return {'loss': loss_car, 'log': {'train_loss': loss_car}}
        
    # validation step
    def validation_step(self, batch, idx):
        car, car_norm, inflow, inflow_norm, revision, revision_norm, \
        transcriptid, embeddings, mask, \
        fin_ratios = batch
        
        # get batch size
        bsz = fin_ratios.size(0)

        # forward
        y_car = self.forward(embeddings, mask, fin_ratios) # (N, 1)

        # compute loss
        loss_car = self.mse_loss(y_car, car_norm.unsqueeze(-1)).unsqueeze(-1) # (1,)

        # logging
        return {'val_loss': loss_car}

    # test step
    def test_step(self, batch, idx):
        car, car_norm, inflow, inflow_norm, revision, revision_norm, \
        transcriptid, embeddings, mask, \
        fin_ratios = batch
        
        # get batch size
        bsz = fin_ratios.size(0)

        # forward
        y_car = self.forward(embeddings, mask, fin_ratios) # (N, 1)

        # compute loss
        loss_car = self.mse_loss(y_car, car_norm.unsqueeze(-1)).unsqueeze(-1) # (1,)

        # logging
        return {'test_loss': loss_car}  

## MTL

In [25]:
# MTL-text-fr-Inf
class CCTransformerMTLInf(CC):
    def __init__(self, hparams):
        # `self.hparams` will be created by super().__init__
        super().__init__(hparams)
        
        # specify model type
        self.model_type = 'TSFM'
        self.target_type = 'car+inf'
        self.feature_type = 'txt+fr'
        self.emb_share = 'hard'
        self.attn_type = 'dotprod'
        self.text_in_dataset = True if self.feature_type!='fr' else False 

        self.n_covariate = 15
        
        # positional encoding
        self.encoder_pos = PositionalEncoding(hparams.d_model, self.hparams.attn_dropout)
        
        # encoder layers for input, expert, nonexpert
        encoder_layers_expert = nn.TransformerEncoderLayer(self.hparams.d_model, self.hparams.n_head_encoder, self.hparams.dff, self.hparams.attn_dropout)
        
        # atten layers
        self.attn_layers_car = nn.Linear(self.hparams.d_model, 1)
        self.attn_dropout_1 = nn.Dropout(self.hparams.attn_dropout)
        
        # Build Encoder
        self.encoder_expert = nn.TransformerEncoder(encoder_layers_expert, self.hparams.n_layers_encoder)
        
        # linear layer to produce final result
        self.linear_car_1 = nn.Linear(self.hparams.d_model, self.hparams.d_model)
        self.linear_car_2 = nn.Linear(self.hparams.d_model, self.hparams.final_tdim)
        self.linear_car_3 = nn.Linear(self.hparams.final_tdim+self.n_covariate, self.hparams.final_tdim+self.n_covariate)
        self.linear_car_4 = nn.Linear(self.hparams.final_tdim+self.n_covariate, self.hparams.final_tdim+self.n_covariate)
        self.linear_car_5 = nn.Linear(self.hparams.final_tdim+self.n_covariate, 1)
        
        self.linear_inflow = nn.Linear(self.hparams.final_tdim, 1)
        # self.linear_revision = nn.Linear(hparam.final_tdim, 1)
        
        # dropout for final fc layers
        self.final_dropout_1 = nn.Dropout(self.hparams.dropout)
        self.final_dropout_2 = nn.Dropout(self.hparams.dropout)
        self.final_dropout_3 = nn.Dropout(self.hparams.dropout)
        
        # layer normalization
        if self.hparams.normalize_layer:
            self.layer_norm = nn.LayerNorm(self.hparams.final_tdim+self.n_covariate)
            
        # batch normalization
        if self.hparams.normalize_batch:
            self.batch_norm = nn.BatchNorm1d(self.n_covariate)

    # forward
    def forward(self, inp, src_key_padding_mask, fin_ratios):
        
        bsz, embed_dim = embeddings.size(0), embeddings.size(2)
        
        # if S is longer than max_seq_len, cut
        embeddings = embeddings[:,:self.hparams.max_seq_len,] # (N, S, E)
        src_key_padding_mask = src_key_padding_mask[:,:self.hparams.max_seq_len] # (N, S)
        
        embeddings = embeddings.transpose(0, 1) # (S, N, E)
        
        # positional encoding
        x = self.encoder_pos(embeddings) # (S, N, E)
        
        # encode
        x_expert = self.encoder_expert(x, src_key_padding_mask=src_key_padding_mask).transpose(0,1) # (N, S, E)
        
        # multiply with attn
        x_attn = self.attn_dropout_1(F.softmax(self.attn_layers_car(x_expert), dim=1)) # (N, S, 1)
        x_expert = torch.bmm(x_expert.transpose(-1,-2), x_attn).squeeze(-1) # (N, E)
        
        # mix with covariate
        x_expert = self.final_dropout_1(F.relu(self.linear_car_1(x_expert))) # (N, E)
        x_expert = F.relu(self.linear_car_2(x_expert)) # (N, final_tdim)
        
        # batch normalization
        if self.hparams.normalize_batch:
            fin_ratio = self.batch_norm(fin_ratios)
        
        x_car = torch.cat([x_expert, fin_ratios], dim=-1) # (N, X + final_tdim) where X is the number of covariate (n_covariate)

            
        # final FC
        y_inflow = self.linear_inflow(x_expert)
        # y_revision = self.linear_revision(x_expert)
        
        x_car = self.final_dropout_2(F.relu(self.linear_car_3(x_car))) # (N, X + final_tdim)
        y_car = self.linear_car_5(x_car) # (N,1)
        
        # final output
        return y_car, y_inflow
    
    # traning step
    def training_step(self, batch, idx):
        
        car, transcriptid, embeddings, mask, alpha, car_m1_m1, car_m2_m2, car_m30_m3,\
        sest, sue, numest, sstdest, smedest, \
        mcap, roa, bm, debt_asset, volatility, volume, inflow, revision = batch
        
        # get batch size
        bsz = sue.size(0)
        
        # forward
        y_car, y_inflow = self.forward(embeddings, mask, alpha, car_m1_m1, car_m2_m2, car_m30_m3, sest, sue, numest, sstdest, \
                             smedest, mcap, roa, bm, debt_asset, volatility, volume) # (N, 1)
        
        # compute loss
        loss_car = self.mse_loss(y_car, car.unsqueeze(-1)).unsqueeze(-1) # (1,)
        loss_inflow = self.mse_loss(y_inflow, inflow.unsqueeze(-1)).unsqueeze(-1) # (1,)
        
        loss = loss_car + loss_inflow
        
        # logging
        return {'loss': loss, 'log': {'train_loss': loss}}
        
    # validation step
    def validation_step(self, batch, idx):
        car, transcriptid, embeddings, mask, alpha, car_m1_m1, car_m2_m2, car_m30_m3,\
        sest, sue, numest, sstdest, smedest, \
        mcap, roa, bm, debt_asset, volatility, volume, inflow, revision = batch
        
        # get batch size
        bsz = sue.size(0)

        # forward
        y_car, y_inflow = self.forward(embeddings, mask, alpha, car_m1_m1, car_m2_m2, car_m30_m3, sest, sue, numest, sstdest, \
                             smedest, mcap, roa, bm, debt_asset, volatility, volume) # (N, 1)

        # compute loss
        loss_car = self.mse_loss(y_car, car.unsqueeze(-1)).unsqueeze(-1) # (1,)
        loss_inflow = self.mse_loss(y_inflow, inflow.unsqueeze(-1)).unsqueeze(-1) # (1,)
        loss = loss_car + loss_inflow
        
        # logging
        return {'val_loss': loss}

    # test step
    def test_step(self, batch, idx):
        car, transcriptid, embeddings, mask, alpha, car_m1_m1, car_m2_m2, car_m30_m3,\
        sest, sue, numest, sstdest, smedest, \
        mcap, roa, bm, debt_asset, volatility, volume, inflow, revision = batch
        
        # get batch size
        bsz = sue.size(0)

        # forward
        y_car, y_inflow = self.forward(embeddings, mask, alpha, car_m1_m1, car_m2_m2, car_m30_m3, sest, sue, numest, sstdest, smedest,\
                             mcap, roa, bm, debt_asset, volatility, volume) # (N, 1)

        # compute loss
        loss_car = self.mse_loss(y_car, car.unsqueeze(-1)).unsqueeze(-1) # (1,)
        loss_inflow = self.mse_loss(y_inflow, inflow.unsqueeze(-1)).unsqueeze(-1) # (1,)
        loss = loss_car + loss_inflow

        # logging
        return {'test_loss': loss}  

## run

In [ ]:
# choose Model
Model = CCTransformerMTLInf

# hparams
model_hparams = {
    'preembedding_type': 'all_sbert_roberta_nlistsb_encoded', # key!
    'targets_name': 'f_sue_keydevid_car_finratio_vol_transcriptid_sim_inflow_revision_text', # key!
    'roll_type': '3y',  # key!
    'batch_size': 20,
    'val_batch_size': 4,
    'max_seq_len': 768, 
    'learning_rate': 3e-4,
    'task_weight': 1,
    'normalize_layer': False, # key!
    'normalize_batch': True, # key!

    'n_layers_encoder': 4,
    'n_head_encoder': 8, 
    'd_model': 1024,
    'final_tdim': 1024, 
    'dff': 2048,
    'attn_dropout': 0.1,
    'dropout': 0.5,
    'n_head_decoder': 8} 

train_hparams = {
    # log
    'note': 'temp',
    'row_log_interval': 1,
    'save_top_k': 1,
    'val_check_interval': 0.2,
    'machine': 'yu-workstation',

    # data size
    'overfit_baches': 1,
    'min_epochs': 3,
    'max_epochs': 20,
    'max_steps': None,
    'accumulate_grad_batches': 1,

    # Caution:
    # The check of patience depends on **how often you compute your val_loss** (`val_check_interval`). 
    # Say you check val every N baches, then `early_stop_callback` will compare to your latest N **baches**.
    # If you compute val_loss every N **epoches**, then `early_stop_callback` will compare to the latest N **epochs**.
    'early_stop_patience': 5,

    # Caution:
    # If set to 1, then save ckpt every 1 epoch
    # If set to 0, then save ckpt on every val!!! (if val improves)
    'checkpoint_period': 0}

# delete all existing .ckpt files
refresh_ckpt()

# load split_df
load_split_df(model_hparams['roll_type'])
    
# load targets_df
load_targets(model_hparams['targets_name'])

# load preembeddings
load_preembeddings(model_hparams['preembedding_type'])
    
# loop over 24!
for window_i in range(len(split_df))[:1]:

    # train one window
    train_one(Model, window_i, model_hparams, train_hparams, preembeddings, targets_df, split_df)

# Predict

In [18]:
# test on one batch
def test_step_fr(model, batch):
    # get input
    docid, car, inp = batch

    # forward
    y_car = model(inp).item() # (N, 1)

    return y_car, docid[0]
    
    
def test_step_text(model, batch):
    car, transcriptid, embeddings, mask, alpha, car_m1_m1, car_m2_m2, car_m30_m3,\
    sest, sue, numest, sstdest, smedest, \
    mcap, roa, bm, debt_asset, volatility, volume = batch

    # forward
    y_car = model(embeddings, mask).item() # (N, 1)
    transcriptid = transcriptid.int().item()
    docid = targets_df.loc[targets_df.transcriptid==transcriptid].docid.iloc[0]

    return y_car, docid

def test_step_text_fr(model, batch):
    car, transcriptid, embeddings, mask, alpha, car_m1_m1, car_m2_m2, car_m30_m3,\
    sest, sue, numest, sstdest, smedest, \
    mcap, roa, bm, debt_asset, volatility, volume = batch

    # forward
    y_car = model(embeddings, mask, alpha, car_m1_m1, car_m2_m2, car_m30_m3, sest, sue, numest, sstdest, \
                  smedest, mcap, roa, bm, debt_asset, volatility, volume).item() # (N, 1)
    transcriptid = transcriptid.int().item()
    docid = targets_df.loc[targets_df.transcriptid==transcriptid].docid.iloc[0]

    return y_car, docid

# test on one window
def test_one_window(model, dataloader):
    # select test_step
    if type(model) in [CCMLP]:
        test_step = test_step_fr
    elif type(model) in [CCTransformerSTLTxtFr]:
        test_step = test_step_text_fr
    # elif type(model) in [CCTransformerSTLTxtFr]:
    #    test_step = test_step_text_fr
    
    ys = []
    for i, batch in enumerate(tqdm(dataloader)):
        ys.append(test_step(model, batch))
        # if i>=2: break
    return pd.DataFrame(ys, columns=['y_car', 'docid'])

# get prediction
def predict_car():
    global hparams
    hparams = Namespace(**hparams)

    # get roll_type
    hparams.roll_type = hparams.ckpt_folder.split('/')[-1].split('-')[0]
    
    # load split_df
    load_split_df(hparams.roll_type)

    # load targets_df
    load_targets(hparams.targets_name)

    # load preembedding
    load_preembeddings(hparams.preembedding_type)
    
    car_prediction = []
    for i, name in enumerate(sorted(os.listdir(f'D:/Checkpoints/earnings-call/{hparams.ckpt_folder}'))):
        # if i>=1: break
            
        if name.endswith('.ckpt'):

            # get window
            window = re.search(r'roll-\d{2}', name).group()

            # load model
            model = hparams.Model.load_from_checkpoint(f'D:/Checkpoints/earnings-call/{hparams.ckpt_folder}/{name}')

            # get testloader
            model.prepare_data()
            test_dataloader = model.test_dataloader()
            model.freeze()
            
            # predict
            y = test_one_window(model, test_dataloader)
            y['window'] = window
            y['roll_type'] = hparams.roll_type

            # append to ys
            car_prediction.append(y)
    car_prediction = pd.concat(car_prediction)
    
    car_prediction.reset_index().to_feather(f'data/{hparams.save_name}.feather')
    return car_prediction

In [19]:
hparams = {
    'Model': CCTransformerSTLTxtFr, # key!
    'ckpt_folder': '3y-TSFM-txt-fr', # key!
    'save_name': 'car_pred_tsfm_txt_fr', # key!

    'targets_name': 'f_sue_keydevid_car_finratio_vol_transcriptid_sim_text',
    'preembedding_type': 'all_sbert_roberta_nlistsb_encoded',
}

# get car_ranking_predict
car_pred_mlp = predict_car()

# car_pred_mlp.reset_index().to_feather('data/car_pred_mlp.feather')

Current window: roll-01 (3y) 
(train: 2008-01-01 to 2010-12-31) (test: 2011-01-01 to 2011-03-31)



Current window: roll-02 (3y) 
(train: 2008-04-01 to 2011-03-31) (test: 2011-04-01 to 2011-06-30)



Current window: roll-03 (3y) 
(train: 2008-07-01 to 2011-06-30) (test: 2011-07-01 to 2011-09-30)



Current window: roll-04 (3y) 
(train: 2008-10-01 to 2011-09-30) (test: 2011-10-01 to 2011-12-31)



Current window: roll-05 (3y) 
(train: 2009-01-01 to 2011-12-31) (test: 2012-01-01 to 2012-03-31)


...


Current window: roll-28 (3y) 
(train: 2014-10-01 to 2017-09-30) (test: 2017-10-01 to 2017-12-31)



Current window: roll-29 (3y) 
(train: 2015-01-01 to 2017-12-31) (test: 2018-01-01 to 2018-03-31)



Current window: roll-30 (3y) 
(train: 2015-04-01 to 2018-03-31) (test: 2018-04-01 to 2018-06-30)



Current window: roll-31 (3y) 
(train: 2015-07-01 to 2018-06-30) (test: 2018-07-01 to 2018-09-30)



Current window: roll-32 (3y) 
(train: 2015-10-01 to 2018-09-30) (test: 2018-10-01 to 2018-12-31)


In [101]:
car_pred_mlp.loc[car_pred_mlp.docid=='031122-2011-01-10']

,y_car,docid,window,roll_type
224,15.298223,031122-2011-01-10,roll-01,3y


# Comet Log

**Task:**
- modify experiment attributes

In [17]:
query = ((comet_ml.api.Metric('test_rmse')!=None) &
         (comet_ml.api.Parameter('model_type')!=None))

exps = comet_ml.api.API().query('amiao', 'earnings-call', query, archived=False)

for exp in exps:
#     roll_type = exp.get_parameters_summary('roll_type').valueCurrent
#     print(roll_type)
    exp.log_parameter('model_type', 'TSFM')

{'name': 'roll_type', 'valueMax': '3y', 'valueMin': '3y', 'valueCurrent': '3y', 'timestampMax': 1590733074542, 'timestampMin': 1590733074542, 'timestampCurrent': 1590733074542, 'editable': False}
{'name': 'roll_type', 'valueMax': '3y', 'valueMin': '3y', 'valueCurrent': '3y', 'timestampMax': 1590733505700, 'timestampMin': 1590733505700, 'timestampCurrent': 1590733505700, 'editable': False}
{'name': 'roll_type', 'valueMax': '3y', 'valueMin': '3y', 'valueCurrent': '3y', 'timestampMax': 1590734234158, 'timestampMin': 1590734234158, 'timestampCurrent': 1590734234158, 'editable': False}
{'name': 'roll_type', 'valueMax': '3y', 'valueMin': '3y', 'valueCurrent': '3y', 'timestampMax': 1590734869878, 'timestampMin': 1590734869878, 'timestampCurrent': 1590734869878, 'editable': False}
{'name': 'roll_type', 'valueMax': '3y', 'valueMin': '3y', 'valueCurrent': '3y', 'timestampMax': 1590735610283, 'timestampMin': 1590735610283, 'timestampCurrent': 1590735610283, 'editable': False}
{'name': 'roll_type'

**Task:**
- download comet log

In [3]:
%%time
# exps = comet_ml.api.API().query('amiao', 'earnings-call', comet_ml.api.Metric('test_rmse') != None, archived=False)

exps = comet_ml.api.API().query('amiao', 'earnings-call', comet_ml.api.Metric('test_rmse') != None, archived=False)

log_comet = []
for exp in exps:
    # get parameter
    log = {param['name']:param['valueCurrent'] for param in exp.get_parameters_summary()}
    
    # get metrics
    log['test_rmse'] = exp.get_metrics('test_rmse')[0]['metricValue']
    
    # get metadat
    log = {**log, **exp.get_metadata()}
    
    # delete useless params
    for key in ['checkpoint_path']:
        log.pop("key", None)
    log_comet.append(log)
    
log_comet = pd.DataFrame(log_comet)
log_comet.to_feather('data/comet_log.feather')

Wall time: 49.2 s


# Backup

## `Dataloader`

In [10]:
class _RepeatSampler(object):
    """ Sampler that repeats forever.

    Args:
        sampler (Sampler)
    """

    def __init__(self, sampler):
        self.sampler = sampler

    def __iter__(self):
        while True:
            yield from iter(self.sampler)

class FastDataLoader(torch.utils.data.dataloader.DataLoader):

    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        object.__setattr__(self, 'batch_sampler', _RepeatSampler(self.batch_sampler))
        self.iterator = super().__iter__()

    def __len__(self):
        return len(self.batch_sampler.sampler)

    def __iter__(self):
        for i in range(len(self)):
            yield next(self.iterator)

### `val` is after `train`

In [ ]:
# Dataset: Txt + Fin-ratio
class CCDataset(Dataset):
    
    def __init__(self, split_window, split_type, text_in_dataset, roll_type, print_window, valid_transcriptids=None, transform=None):
        '''
        Args:
            preembeddings (from globals): list of embeddings. Each element is a tensor (S, E) where S is number of sentences in a call
            targets_df (from globals): DataFrame of targets variables.
            split_df (from globals):
            split_window: str. e.g., "roll-09"
            split_type: str. 'train' or 'test'
            text_only: only output CAR and transcripts if true, otherwise also output financial ratios
            transcriptids: list. If provided, only the given transcripts will be used in generating the Dataset. `transcriptids` is applied **on top of** `split_window` and `split_type`
        '''

        self.text_in_dataset = text_in_dataset
        
        # decalre data as globals so don't   need to create/reload
        global preembeddings, targets_df, split_df
        
        # get split dates from `split_df`
        _, train_start, train_end, val_start, val_end, test_start, test_end, _ = tuple(split_df.loc[(split_df.window==split_window) & (split_df.roll_type==roll_type)].iloc[0])
        # print current window
        if print_window:
            print(f'Current window: {split_window} ({roll_type}) \n(train: {train_start} to {train_end}) (val: {val_start} to {val_end}) (test: {test_start} to {test_end})')
        
        train_start = datetime.strptime(train_start, '%Y-%m-%d').date()
        train_end = datetime.strptime(train_end, '%Y-%m-%d').date()
        val_start = datetime.strptime(val_start, '%Y-%m-%d').date()
        val_end = datetime.strptime(val_end, '%Y-%m-%d').date()
        test_start = datetime.strptime(test_start, '%Y-%m-%d').date()
        test_end = datetime.strptime(test_end, '%Y-%m-%d').date()
        
        # select valid transcriptids (preemb_keys) according to split dates 
        if split_type=='train':
            transcriptids = targets_df[targets_df.ciq_call_date.between(train_start, train_end)].transcriptid.tolist()
        if split_type=='val':
            transcriptids = targets_df[targets_df.ciq_call_date.between(val_start, val_end)].transcriptid.tolist()
        elif split_type=='test':
            transcriptids = targets_df[targets_df.ciq_call_date.between(test_start, test_end)].transcriptid.tolist()

        self.valid_preemb_keys = set(transcriptids).intersection(set(preembeddings.keys()))
        
        if valid_transcriptids is not None:
            self.valid_preemb_keys = self.valid_preemb_keys.intersection(set(valid_transcriptids))
        
        # self attributes
        self.targets_df = targets_df
        self.preembeddings = preembeddings
        self.transform = transform
        self.sent_len = sorted([(k, preembeddings[k].shape[0]) for k in self.valid_preemb_keys], key=itemgetter(1))
        self.train_start = train_start
        self.train_end = train_end
        self.test_start = test_start
        self.test_end = test_end
        self.n_samples = len(self.sent_len)
        self.split_window = split_window
        self.split_type = split_type
        
    def __len__(self):
        return (len(self.valid_preemb_keys))
    
    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()
            
        transcriptid = self.sent_len[idx][0]
        targets = self.targets_df[self.targets_df.transcriptid==transcriptid].iloc[0]
        
        # inputs: preembeddings
        embeddings = self.preembeddings[transcriptid]
        
        # all of the following targests are
        # of type `numpy.float64`
        sue = targets.sue
        sest = targets.sest
        car_0_30 = targets.car_0_30
        
        alpha = targets.alpha
        volatility = targets.volatility
        mcap = targets.mcap/1e6
        bm = targets.bm
        roa = targets.roa
        debt_asset = targets.debt_asset
        numest = targets.numest
        smedest = targets.smedest
        sstdest = targets.sstdest
        car_m1_m1 = targets.car_m1_m1
        car_m2_m2 = targets.car_m2_m2
        car_m30_m3 = targets.car_m30_m3
        volume = targets.volume
        
        if self.text_in_dataset:
            return car_0_30, transcriptid, embeddings, alpha, car_m1_m1, car_m2_m2, car_m30_m3, \
                   sest, sue, numest, sstdest, smedest, \
                   mcap, roa, bm, debt_asset, volatility, volume
        else:
            return torch.tensor(car_0_30,dtype=torch.float32), \
                   torch.tensor([alpha, car_m1_m1, car_m2_m2, car_m30_m3, sest, sue, numest, sstdest, smedest, mcap, roa, bm, debt_asset, volatility, volume], dtype=torch.float32)
    
